In [19]:
from helpers import linear_helper as linh, nn_helper as nneh, lgb_helper as lgbh
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import os
import pickle

In [20]:
model1 = nneh.load_model_nn('../models/model_char_cnn_emb.hdf5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 150)          15600     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 150, 150)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 148, 150)          67650     
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 148, 150)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 148, 150)          600       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 146, 150)          67650     
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 146, 150)          0         
__________

In [21]:
model2 = nneh.load_model_nn('../models/model_lstm_channel_classification.hdf5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 300)          15000000  
_________________________________________________________________
dropout_1 (Dropout)          (None, 150, 300)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 150, 150)          270600    
_________________________________________________________________
lstm_2 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dropout_2 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 150)               22650     
_________________________________________________________________
dropout_3 (Dropout)          (None, 150)               0         
__________

In [22]:
model3 = lgbh.load_model('../models/lightgbm_model.txt')

In [23]:
model4 = linh.load_model('../models/logregression_channels.pickle')

In [25]:
train_text, val_text, _, val_labels = lgbh.load_data_gbm('../data/train_set.csv')
vectorizer = TfidfVectorizer(analyzer = 'char', max_features = 1000000, ngram_range = (1, 4))
train_matrix = vectorizer.fit_transform(train_text)

In [26]:
mappings = {
    'career': 0,
    'theory_and_practice': 1,
    'deep_learning': 2,
    'lang_python': 3,
    '_meetings': 4,
    'kaggle_crackers': 5,
    'big_data': 6,
    'lang_r': 7,
    'nlp': 8,
    'welcome': 9,
    'datasets': 10,
    'bayesian': 11
}

In [27]:
MAX_SEQUENCE_LENGTH = 150
def classify_text(text_arr):

    val_matrix = vectorizer.transform(text_arr)
    lgb_class = model3.predict(val_matrix, num_iteration=model3.best_iteration)

    decision_boundary = model4.decision_function(text_arr)
    linear_class = np.exp(decision_boundary) / np.sum(np.exp(decision_boundary), axis=1)[:, np.newaxis]

    vocab, _ = nneh.create_vocab_set()
    text_val = nneh.pad_sequences(nneh.text2sequence(text_arr, vocab), maxlen=150, value=0)

    nn1_class = model1.predict(text_val)

    index = np.argmax(lgb_class*0.5 + linear_class*0.2 + nn1_class*0.3, axis=1)[0]
    for key in mappings:
        if mappings[key] == index:
            return key

In [28]:
classify_text(['поламался хадуп'])

'big_data'

In [10]:
classify_text(['какая зп в германии'])

'career'

In [11]:
classify_text(['не знаю в какой канал лучше написать, попробую сюда'])

'deep_learning'

In [12]:
classify_text(['для визуализации R, конечно получше будет, но и так сойдет!'])

'lang_python'

In [13]:
classify_text(['ggplot rules!!'])

'lang_r'

In [14]:
classify_text(['matplotlib - топ, ggplot ваще не сранить!!'])

'lang_python'

In [15]:
classify_text(['кто-то пробовал с пандаса на диплайр свитчиться?'])

'lang_python'

In [36]:
v = np.argmax(lgb_class*i + linear_class*j + nn1_class*k + nn2_class*(1 - (i+j+k)), axis=1)
val_labels == v

NameError: name 'lgb_class' is not defined

In [ ]:
for row in (lgb_class*0.5 + linear_class*0.3 + nn1_class*0.2):
    print(np.sum(row/np.sum(row)))
            

In [33]:
import pandas as pd

In [34]:
df = pd.DataFrame.from_dict(val_set, 'index')

NameError: name 'val_set' is not defined

In [ ]:
df.columns = ['val']

In [ ]:
df.sort_values('val', ascending=False)

In [29]:
DIR_TRAIN = '../data'
data = pd.read_csv(os.path.join(DIR_TRAIN, 'train_set.csv'), usecols=range(1,11), parse_dates=['timestamp', 'thread_timestamp'])

In [30]:
import re
from fastnumbers import isfloat, isint
temp_data = data[
        data.channel.isin(['career'])
    ]
temp_data.text = temp_data.text.astype(str)\
    .apply(lambda x: re.sub('(<\S+>:?)|(\s?:\S+:\s?)|(&gt;)|([\w\.]*@[\w\.]*)|(&\w+;)', ' ', x))\
    .apply(lambda x: re.sub('\d+', ' ', x))\
    .apply(lambda x: re.sub('\s+', ' ', x))
temp_data = temp_data[['channel', 'text']].reset_index()[['channel', 'text']]
temp_data.loc[:,'channel'] = temp_data.channel.map(mappings)
temp_data = temp_data.sort_values('channel').reset_index()[['channel', 'text']]
temp_data = temp_data[~temp_data.text.apply(lambda x: isfloat(x) or isint(x) or len(x) < 20)]

/home/ubuntu/miniconda/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [31]:
temp_data = temp_data[['channel', 'text']].reset_index()[['channel', 'text']]

In [32]:
def unmap(val):
    for key in mappings:
        if mappings[key] == val:
            return key

In [33]:
def classify_data(data_entry):
    if data_entry[2]%1000 == 0:
        print("Processing {0} entry...".format(data_entry[2]))
    label = classify_text([data_entry[1]])
    if label != unmap(data_entry[0]):
        return {'correct': unmap(data_entry[0]), 'predicted': label, 'text': data_entry[1]}
    else:
        pass

In [34]:
lst = temp_data.values.tolist()
for index, item in enumerate(lst):
    lst[index].append(index)

In [35]:
lst

[[0, 'телепортирую обсуждение из а почему именно биотех? ', 0],
 [0,
  ' мне видится, что здоровье, еда, вода и энергия всегда важнее остального. Население Земли растёт и спрос на вышеперечисленное будет расти сильнее хорошей рекомендашки что послушать в )',
  1],
 [0,
  'Про здоровье согласен. Вообще не понимаю, почему различные полеты в космос и тесла автомобили собирают огромные деньги, а стартапы по печатанию органов таких денег не собирают. Я бы занялся такими вещами',
  2],
 [0,
  'Рост населения в посление годы не такой резкий, это больше похоже на выход на плато. Так что ресурсов хватит еще надолго ',
  3],
 [0,
  ' нет саксесс стори и интереса соотв. Ну и инвестрынок спекулятивен по своей сути. Раз быстро не сделаешь денег, лучше не соваться и подождать когда станет хот. А стратегов очень мало из-за отсутствия саксесс стори и соотв рынка хоть какого-то. Кмк как-то так не хочу спорить, но когда-то и самый богатый человек в Мире считал, что всем хватит сколько-то киллобайт RAM) 

In [36]:
def save_obj(obj, name ):
    with open(os.path.join(DIR_TRAIN, '{0}.pkl'.format(name)), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [39]:
for item in lst[:10]:
    print(classify_data(item))

Processing 0 entry...
None
None
None
None
None
None
{'correct': 'career', 'predicted': 'theory_and_practice', 'text': 'Ага, после фиксанья ночных багов за собой и другими начинаешь обкладываться логами, метриками, ранбуками и код ревью.'}
None
None
None


In [ ]:
from multiprocessing.dummy import Pool as ThreadPool 

# make the Pool of workers
pool = ThreadPool(16) 

# open the urls in their own threads
# and return the results
results = pool.map(classify_data, lst)

# close the pool and wait for the work to finish 
pool.close() 
pool.join() 

Processing 0 entry...
Processing 7000 entry...
Processing 4000 entry...
Processing 11000 entry...
Processing 1000 entry...
Processing 8000 entry...
Processing 5000 entry...
Processing 12000 entry...
Processing 2000 entry...
Processing 9000 entry...
Processing 6000 entry...
Processing 3000 entry...
Processing 10000 entry...
Processing 17000 entry...
Processing 14000 entry...
Processing 24000 entry...
Processing 21000 entry...
Processing 18000 entry...
Processing 15000 entry...
Processing 22000 entry...
Processing 19000 entry...
Processing 16000 entry...
Processing 13000 entry...
Processing 23000 entry...
Processing 20000 entry...
Processing 27000 entry...


In [ ]:
results =  [x for x in results if x is not None]


In [ ]:
save_obj(results, 'carreers')

In [48]:
results

[{'correct': 'lang_python',
  'predicted': '_meetings',
  'text': 'Завтра буду доставать тупыми вопросами.'},
 {'correct': 'lang_python',
  'predicted': '_meetings',
  'text': ' рискну упомянуть курс в CSC от Сергея Лебедева: в первых лекциях -- элементарщина, но далее как раз глубина'},
 {'correct': 'lang_python',
  'predicted': 'career',
  'text': ' Глянул сейчас программу занятий его, прям достойно, да'},
 {'correct': 'lang_python',
  'predicted': 'theory_and_practice',
  'text': 'но это классика, я так поняла, вопрос про более экзотические штуки '},
 {'correct': 'lang_python',
  'predicted': 'career',
  'text': 'Тонкости обычно нужны тогда, когда они действительно нужны, главное знать об их существовании. В лутце об этом говорят, что есть хорошо'},
 {'correct': 'lang_python',
  'predicted': 'career',
  'text': 'Да, хотелось бы вот вроде того же Бизли. Лутц хорош, спору нет, но листать книгу в страниц ради тонкостей, которые в ней не подряд идут, а от главы к главе появляются и исче

In [ ]:
for channel in ['career', 'big_data', 'deep_learning', 'hardware', 'kaggle_crackers',
           'lang_python',  'lang_r', 'nlp', 'theory_and_practice', 'welcome', 'bayesian', '_meetings', 'datasets']:
    temp_data = data[
            data.channel.isin([channel])
        ]
    temp_data.text = temp_data.text.astype(str)\
        .apply(lambda x: re.sub('(<\S+>:?)|(\s?:\S+:\s?)|(&gt;)|([\w\.]*@[\w\.]*)|(&\w+;)', ' ', x))\
        .apply(lambda x: re.sub('\d+', ' ', x))\
        .apply(lambda x: re.sub('\s+', ' ', x))
    temp_data = temp_data[['channel', 'text']].reset_index()[['channel', 'text']]
    temp_data.loc[:,'channel'] = temp_data.channel.map(mappings)
    temp_data = temp_data.sort_values('channel').reset_index()[['channel', 'text']]
    temp_data = temp_data[~temp_data.text.apply(lambda x: isfloat(x) or isint(x) or len(x) < 20)]
    temp_data = temp_data[['channel', 'text']].reset_index()[['channel', 'text']]
    
    lst = temp_data.values.tolist()
    for index, item in enumerate(lst):
        lst[index].append(index)
    
    # make the Pool of workers
    pool = ThreadPool(16) 

    # open the urls in their own threads
    # and return the results
    results = pool.map(classify_data, lst)

    # close the pool and wait for the work to finish 
    pool.close() 
    pool.join() 
    results =  [x for x in results if x is not None]
    save_obj(results, channel)

/home/ubuntu/miniconda/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Processing 0 entry...
Processing 7000 entry...
Processing 4000 entry...
Processing 11000 entry...
Processing 1000 entry...
Processing 8000 entry...
Processing 5000 entry...
Processing 12000 entry...
Processing 2000 entry...
Processing 9000 entry...
Processing 6000 entry...
Processing 10000 entry...
Processing 3000 entry...
Processing 17000 entry...
Processing 14000 entry...
Processing 21000 entry...
Processing 24000 entry...
Processing 18000 entry...
Processing 15000 entry...
Processing 22000 entry...
Processing 19000 entry...
Processing 16000 entry...
Processing 13000 entry...
Processing 23000 entry...
Processing 20000 entry...
Processing 27000 entry...
Processing 34000 entry...
Processing 31000 entry...
Processing 28000 entry...
Processing 25000 entry...
Processing 35000 entry...
Processing 32000 entry...
Processing 29000 entry...
Processing 26000 entry...
Processing 36000 entry...
Processing 33000 entry...
Processing 30000 entry...
Processing 37000 entry...
Processing 47000 entry...

In [50]:
1

1